## Python task

In [1]:
from bs4 import BeautifulSoup as Soup
import requests
import pandas as pd

In [2]:
def get_table(URL, file_name):
    
    html = requests.get(URL)
    soup = Soup(html.text, "lxml")
    
    #Extracting rows
    rows = []
    find_table = soup.find("table")
    find_rows = find_table.find_all("tr")
    for row in find_rows:
        column = row.find_all("td")
        column = [row.text.strip() for row in column]
        rows.append(column)
        
    #Extracting titles    
    titles = []
    find_titles = find_table.find_all("th")
    for title in find_titles:
        title = title.text.strip()
        titles.append(title)
        
    #Join all data to DF + export to CSV
    data = pd.DataFrame(rows)
    data.drop(data.index[0], inplace = True)
    data.columns = titles
    data.to_csv("{}.csv".format(file_name))
    return data

get_table("https://worldpopulationreview.com/country-rankings/crime-rate-by-country", "crime_statistics")

,Ranking,Country,Crime Index,2021 Population
1,1,Venezuela,83.76,"28,704,954"
2,2,Papua New Guinea,80.79,"9,119,010"
3,3,South Africa,76.86,"60,041,994"
4,4,Afghanistan,76.31,"39,835,428"
5,5,Honduras,74.54,"10,062,991"
...,...,...,...,...
132,133,Oman,20.34,"5,223,375"
133,134,Isle of Man,19.25,"85,410"
134,135,Taiwan,15.46,"23,855,010"
135,136,United Arab Emirates,15.23,"9,991,089"


## SQL task

In [2]:
import sqlite3 as sq3

In [3]:
data = pd.read_csv("crime_statistics.csv")
db = sq3.connect("crime_statistics_db.db")
data.to_sql("crimes", db, index = False)
cur = db.cursor()

In [6]:
#Count the values in each column

cur.execute('SELECT COUNT("Ranking") AS "Ranking",\
            COUNT("Country") AS "Country",\
            COUNT("Crime_Index") AS "Crime_Index",\
            COUNT("2021 Population") AS "Population"\
            FROM "crimes"')

value_count = pd.DataFrame(cur.fetchall())
value_count.columns = ["Ranking", "Country", "Crime_Index", "Population"]
display(value_count)

,Ranking,Country,Crime_Index,Population
0,136,136,136,136


In [9]:
#Row with the highest value of crime index

cur.execute('SELECT * FROM "crimes"\
            WHERE "Crime Index" = (\
            SELECT MAX("Crime Index")\
            FROM "crimes"\
            )')

highest_pop = pd.DataFrame(cur.fetchall())
highest_pop.columns = ["Row_number", "Ranking", "Country", "Crime_Index", "Population"]
display(highest_pop)

,Row_number,Ranking,Country,Crime_Index,Population
0,1,1,Venezuela,83.76,"28,704,954"


In [10]:
#Row with the lowest value of crime index

cur.execute('SELECT * FROM "crimes"\
            WHERE "Crime Index" = (\
            SELECT MIN("Crime Index")\
            FROM "crimes"\
            )')

highest_pop = pd.DataFrame(cur.fetchall())
highest_pop.columns = ["Row_number", "Ranking", "Country", "Crime_Index", "Population"]
display(highest_pop)

,Row_number,Ranking,Country,Crime_Index,Population
0,136,137,Qatar,12.13,"2,930,528"


In [4]:
# merge the aboved rows to 1 new table named "merged_rows"

cur.execute('CREATE TABLE "merged_rows"\
            ("Row_number", "Ranking", "Country", "Crime_Index", "2021_Population")')

cur.execute('INSERT INTO "merged_rows"\
            SELECT * FROM "crimes" WHERE "Crime Index" = (SELECT MAX("Crime Index") FROM "crimes")')

cur.execute('INSERT INTO "merged_rows"\
            SELECT * FROM "crimes" WHERE "Crime Index" = (SELECT MIN("Crime Index") FROM "crimes")')

cur.execute('SELECT * FROM "merged_rows"')


merged_rows = pd.DataFrame(cur.fetchall())
merged_rows.columns = ["Row_number", "Ranking", "Country", "Crime_Index", "Population"]
display(merged_rows)

,Row_number,Ranking,Country,Crime_Index,Population
0,1,1,Venezuela,83.76,"28,704,954"
1,136,137,Qatar,12.13,"2,930,528"
